In [1]:
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer

In [76]:
df_all = pd.read_pickle(
    "/Users/vigadam/Dropbox/My Mac (MacBook-Air.local)/Documents/github/media_network/media_data/clean_text/2020/all_site_2020.pkl",
)

In [77]:
df_all.loc[
    df_all["tags"].apply(
        lambda x: "koronavírus magyarországon" in x if type(x) == str else False
    ),
    "page",
].value_counts()
# check for covid benchmark study!!

Index    3168
24.hu      14
Name: page, dtype: int64

1. filter for index, origo, 24.hu

In [78]:
df_all["page"] = df_all["page"].str.lower().str.replace(".", "", regex=True)

In [79]:
df = df_all.copy()

In [80]:
df["page"].value_counts()

origo    57575
index    52676
mno      45155
24hu     37450
444      15978
888      15459
Name: page, dtype: int64

2. filter for section belfold

In [81]:
df = df.assign(
    category=lambda x: x["category"]
    .str.lower()
    .replace("á", "a")
    .replace("ö", "o")
    .replace("ü", "u")
    .replace("é", "e")
    .replace("itthon", "belfold")
    .replace("belföld", "belfold")
    .replace("politika", "belfold")
    .replace("ketharmad", "belfold")
    .replace("egészségügy", "belfold")
    .replace("járvány", "belfold")
    .replace("oktatás", "belfold")
    .replace("budapest", "belfold")
    .replace("koronavírus", "belfold")
).loc[lambda x: x["category"] == "belfold"]

In [82]:
df["page"].value_counts()

origo    16816
index    16386
mno      13340
24hu     12222
888       5722
444       5246
Name: page, dtype: int64

3. drop None titles

In [83]:
df = df.dropna(subset=["title"])

In [90]:
df = df.loc[~df["title"].apply(lambda x: "videó" in x if type(x) == str else False)]

In [91]:
df["page"].value_counts()

index    16320
origo    15620
mno      13078
24hu     12035
888       5683
444       5228
Name: page, dtype: int64

In [92]:
#df.to_pickle("all_sites_belfold.pkl")

In [93]:
df = pd.read_pickle("all_sites_belfold.pkl")
df["page"].value_counts()

index    16320
origo    15620
mno      13078
24hu     12035
888       5683
444       5228
Name: page, dtype: int64

In [94]:
main_pages = ["index","origo","24hu"]

In [95]:
df_main = df.loc[lambda x: x["page"].isin(main_pages)]

In [96]:
df = df_main.copy()

4. check  in index, origo, 24

In [97]:
# some index obs has no tags, but link has coronavirus, tag == koronavírus
df.loc[
    (
        (df["link"].apply(lambda x: "koronavirus" in x if type(x) == str else False))
        & (df["page"] == "index")
        & (df["tags"].isnull())
    ),
    "tags",
] = "koronavírus"

In [52]:
corpus = (
    df["tags"]
    .str.replace("[", "", regex=True)
    .str.replace("]", "", regex=True)
    .str.replace("'", "", regex=True)
    .str.replace(",", "", regex=True)
    .dropna()
)

vectorizer = CountVectorizer()
X = vectorizer.fit_transform(corpus)

dtm = pd.DataFrame(X.toarray(), columns=vectorizer.get_feature_names())

dtm.sum().sort_values(ascending=False).head(25)

koronavírus       15006
baleset            2675
magyarországon     2154
rendőrség          2028
budapest           1962
krimi              1833
orbán              1721
bűnügy             1614
viktor             1602
időjárás           1523
járvány            1367
gergely            1288
péter              1238
magyarország       1114
karácsony          1027
gyilkosság          933
lászló              919
operatív            868
törzs               868
magyar              817
kínai               716
halálos             713
fidesz              683
istván              663
megye               657
dtype: int64

In [53]:
vectorizer = CountVectorizer(ngram_range = (2,2))
X = vectorizer.fit_transform(corpus)

dtm = pd.DataFrame(X.toarray(), columns=vectorizer.get_feature_names())

dtm.sum().sort_values(ascending=False).head(25)

koronavírus magyarországon    2153
orbán viktor                  1590
koronavírus koronavírus       1159
karácsony gergely              895
operatív törzs                 865
kínai koronavírus              709
szijjártó péter                611
koronavírus járvány            566
halálos baleset                513
koronavírus magyarország       506
müller cecília                 448
gyurcsány ferenc               376
covid 19                       358
kásler miklós                  338
gulyás gergely                 310
demokratikus koalíció          292
magyarországon koronavírus     270
európai unió                   265
varga judit                    265
testi sértés                   263
járvány koronavírus            260
heti tv                        258
koronavírus covid              232
novák katalin                  230
soros györgy                   209
dtype: int64

5. drop selected tags

In [98]:
df["krimi"] = df["tags"].apply(lambda x: "krimi" in x if type(x) == str else False)
df["időjárás"] = df["tags"].apply(
    lambda x: "időjárás" in x if type(x) == str else False
)
df["baleset"] = df["tags"].apply(lambda x: "baleset" in x if type(x) == str else False)
df["rendőrség"] = df["tags"].apply(
    lambda x: "rendőrség" in x if type(x) == str else False
)
df["bűnügy"] = df["tags"].apply(lambda x: "bűnügy" in x if type(x) == str else False)
df["gyilkosság"] = df["tags"].apply(lambda x: "gyilkosság" in x if type(x) == str else False)
df["testi_sertes"] = df["tags"].apply(
    lambda x: "testi sértés" in x if type(x) == str else False
)
df["migráns"] = df["tags"].apply(lambda x: "migráns" in x if type(x) == str else False)
df["drop"] = df.filter(
    ["krimi", "időjárás", "baleset", "rendőrség", "bűnügy", "testi_sertes","gyilkosság"]
).sum(axis=1)

# BUT! keep those, that has migráns tag

df.loc[lambda x: (x["drop"] != 0) & (x["migráns"]) == 1,"drop"] = 0

In [99]:
#drop articles based on selected tags
print(df.loc[lambda x: x["drop"] == 0,"page"].value_counts())
df = df.loc[lambda x: (x["drop"] == 0)]

index    14443
origo    11079
24hu      9055
Name: page, dtype: int64


6. create koronavírus category

In [100]:
df["korona"] = df["tags"].apply(lambda x: "koronavírus" in x if type(x) == str else False)

In [101]:
df_covid = df.loc[df["korona"] == 1]

In [102]:
df_covid["page"].value_counts()

index    7176
24hu     2742
origo    2565
Name: page, dtype: int64

In [103]:
def stratified_sample(df, category, sample_size):
    return (
        df.groupby(category, group_keys=False)
        .apply(lambda x: x.sample(int(np.rint(sample_size * len(x) / len(df)))))
        .sample(frac=1)
        .reset_index(drop=True)
    )

In [104]:
# create stratified sample
np.random.seed(42)
N = 1000
df_covid_sample = (
    df_covid.groupby("page", group_keys=False)
    .apply(lambda x: x.sample(int(np.rint(N * len(x) / len(df_covid)))))
    .sample(frac=1)
    .reset_index(drop=True)
)

In [105]:
df_covid_sample["page"].value_counts(normalize=True)

index    0.575
24hu     0.220
origo    0.205
Name: page, dtype: float64

In [106]:
covid_sample = (
    df_covid_sample.filter(["title"])
    .rename(columns={"title": "covid-title"})
    .to_dict("records")
)

with open("json-files/news-headlines-covid.json", "w", encoding="utf8") as outfile:
    json.dump(covid_sample, outfile, ensure_ascii=False)

df_covid_sample.to_pickle("sample_dfs/covid_sample.pkl")

pd.DataFrame(
    ["${" + "e://Field/{i}.covid-title".format(i=i) + "}" for i in range(0, N)]
).reset_index().to_excel("lm_excels/loop_and_merge_covid.xlsx", header=False)

7. drop covid cat

In [107]:
df = df.loc[df["korona"] == 0]

In [108]:
df["page"].value_counts()

origo    8514
index    7267
24hu     6313
Name: page, dtype: int64

In [109]:
np.random.seed(42)
sample_size = 1000

for page in main_pages:

    # create sample frame
    sample_df = (
        df.filter(["page", "title"])
        .loc[lambda x: x["page"] == page]
        .dropna()
        .sample(1000)
        .reset_index()
    )
    # save sample frame
    sample_df.to_pickle("sample_dfs/{page}_sample.pkl".format(page=page))

    # create sample json
    sample = (
        sample_df.rename(columns={"title": "{page}-title".format(page=page)})
        .filter(["{page}-title".format(page=page)])
        .to_dict("records")
    )
    # save sample json
    with open(
        "json-files/news-headlines-{}.json".format(page), "w", encoding="utf8"
    ) as outfile:
        json.dump(sample, outfile, ensure_ascii=False)

    # save lm excel
    pd.DataFrame(
        [
            "${" + "e://Field/{i}.{page}-title".format(page=page, i=i) + "}"
            for i in range(0, N)
        ]
    ).reset_index().to_excel(
        "lm_excels/loop_and_merge_{page}.xlsx".format(page=page), header=False
    )

### 8. create crosscheck table

In [110]:
df = pd.read_pickle("all_sites_belfold.pkl")
# some index obs has no tags, but link has coronavirus, tag == koronavírus
df.loc[
    (
        (df["link"].apply(lambda x: "koronavirus" in x if type(x) == str else False))
        & (df["page"] == "index")
        & (df["tags"].isnull())
    ),
    "tags",
] = "koronavírus"

In [111]:
df["krimi"] = df["tags"].apply(lambda x: "krimi" in x if type(x) == str else False)
df["időjárás"] = df["tags"].apply(
    lambda x: "időjárás" in x if type(x) == str else False
)
df["baleset"] = df["tags"].apply(lambda x: "baleset" in x if type(x) == str else False)
df["rendőrség"] = df["tags"].apply(
    lambda x: "rendőrség" in x if type(x) == str else False
)
df["bűnügy"] = df["tags"].apply(lambda x: "bűnügy" in x if type(x) == str else False)
df["gyilkosság"] = df["tags"].apply(lambda x: "gyilkosság" in x if type(x) == str else False)
df["testi_sertes"] = df["tags"].apply(
    lambda x: "testi sértés" in x if type(x) == str else False
)
df["migráns"] = df["tags"].apply(lambda x: "migráns" in x if type(x) == str else False)
df["drop"] = df.filter(
    ["krimi", "időjárás", "baleset", "rendőrség", "bűnügy", "testi_sertes","gyilkosság"]
).sum(axis=1)

# BUT! keep those, that has migráns tag

df.loc[lambda x: (x["drop"] != 0) & (x["migráns"]) == 1,"drop"] = 0

df = df.loc[lambda x: (x["drop"] == 0)]

In [112]:
df["korona"] = df["tags"].apply(lambda x: "koronavírus" in x if type(x) == str else False)

df = df.loc[df["korona"] == 0]

In [113]:
np.random.seed(42)
crosscheck_sample_df = stratified_sample(df, "page", 100)
crosscheck_sample_df.to_pickle("sample_dfs/crosscheck_sample.pkl")

crosscheck_sample = (
    crosscheck_sample_df.filter(["title"])
    .rename(columns={"title": "crosscheck-title"})
    .to_dict("records")
)

with open("json-files/news-headlines-crosscheck.json", "w", encoding="utf8") as outfile:
    json.dump(crosscheck_sample, outfile, ensure_ascii=False)

pd.DataFrame(
    [
        "${" + "e://Field/{i}.crosscheck-title".format(i=i) + "}"
        for i in range(0, crosscheck_sample_df.shape[0])
    ]
).reset_index().to_excel("lm_excels/loop_and_merge_crosscheck.xlsx", header=False)

9. 1000 sample from other sites

In [114]:
np.random.seed(42)
N = 1000

others_sample_df = stratified_sample(df.loc[~df["page"].isin(main_pages)], "page", N)
others_sample_df.to_pickle("sample_dfs/others_sample.pkl")

others_sample = (
    crosscheck_sample_df.filter(["title"])
    .rename(columns={"title": "other-title"})
    .to_dict("records")
)

with open("json-files/news-headlines-others.json", "w", encoding="utf8") as outfile:
    json.dump(others_sample, outfile, ensure_ascii=False)

pd.DataFrame(
    ["${" + "e://Field/{i}.others-title".format(i=i) + "}" for i in range(0, N)]
).reset_index().to_excel("lm_excels/loop_and_merge_others.xlsx", header=False)

In [ ]:
import ast

In [ ]:
ast.literal_eval()

In [ ]:
results = pd.read_csv("Ádám+survey+try_April+14,+2021_05.18.csv")


In [ ]:
results.T

In [ ]:
results.T.loc[lambda x: x[1] == "995.title-24hu"]

In [ ]:
results = pd.read_csv("Ádám+survey+try_April+14,+2021_05.18.csv")
results = results.T.assign(
    ImportId=lambda x: x[1].apply(ast.literal_eval).apply(lambda x: x["ImportId"],
                                                         )
)

In [ ]:
question_id = results.loc[
    results["ImportId"].apply(lambda x: 1 if "title" in x else None).dropna().index
].filter([2]).rename(columns={2:"title"})

In [ ]:
answers = (
    results.loc[
        results[0].apply(lambda x: 1 if "${e://Field/" in x else None).dropna().index
    ]
    .drop(columns=["ImportId"])
    .assign(
        Id=lambda x: x[0]
        .str.split("}")
        .apply(lambda x: x[0])
        .str.split("/")
        .apply(lambda x: x[-1])
    ).set_index("Id").iloc[:,2:]
)

In [ ]:
question_id

In [ ]:
answers

In [ ]:
pd.merge(question_id.reset_index(),answers.reset_index(),left_on = "index",right_on= "Id")

In [ ]:
results.loc[lambda x: x["ImportId"] == "484.title-index"]

In [ ]:
results.loc[
    lambda x: x[2]
    == "Sokan úgy érzik, jóval nagyobb mértékben drágult az élet, mint a kimutatott adat"
]

In [ ]:
results  = results.assign(head=lambda x: x["ImportId"].str.split("_").apply(lambda x: x[1]))

In [ ]:
results["head"]

In [ ]:
results

In [ ]:
results["ImportId"].apply(lambda x: x["ImportId"])